In [1]:
#warnings.filterwarnings("ignore")  # 忽略警告
from numpy import loadtxt
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import *
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.svm import LinearSVC,SVC,NuSVC
from sklearn.linear_model import ElasticNetCV, LassoCV, LogisticRegressionCV,RidgeClassifierCV,  LogisticRegression, Lasso, RidgeClassifier, SGDClassifier, PassiveAggressiveClassifier,Perceptron
from sklearn.naive_bayes import BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import csv
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
#import shap

# 读取数据

In [142]:
random_num = 1
data = "./data/20240820 对数加一训练集和测试集.xlsx"

# 载入数据集
dataset = pd.DataFrame(pd.read_excel(data, sheet_name="训练集"))
x_train = dataset[dataset.columns[1:]]
y_train = dataset[['Group']].values.ravel()

dataset_test = pd.DataFrame(pd.read_excel(data, sheet_name="测试集"))
x_test = dataset_test[dataset_test.columns[1:]]
y_test = dataset_test[['Group']].values.ravel()


In [143]:
feature_list = ['DBIL', 'TBIL', 'AST', 'D-Dimer', 'PCT', 'CRE', 'Urea', 'LDH', 'IBIL', 'PT']

In [154]:
feature_list = feature_list[0:7]
x_train = x_train[feature_list]
x_test = x_test[feature_list]

In [5]:
def predict_with_model(model, X, threshold=0.5):
    """使用模型进行预测，并根据模型类型选择预测方法。"""
    if hasattr(model, 'decision_function'):
        y_pred = model.decision_function(X)
        y_pred_label = (y_pred > 0).astype(int)
    else:
        y_pred = model.predict_proba(X)[:, 1]
        y_pred_label = (y_pred > threshold).astype(int)
    return y_pred, y_pred_label

def evaluate_model(model, x_train, y_train, x_test, y_test):
    """在训练集和测试集上评估模型。"""
    # 使用最好的模型进行训练集预测
    y_train_pred, y_train_pred_label = predict_with_model(model, x_train)
    train_auc = roc_auc_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred_label)
    train_acc = accuracy_score(y_train, y_train_pred_label)
    
    # 在测试集上进行预测
    y_test_pred, y_test_pred_label = predict_with_model(model, x_test)
    test_auc = roc_auc_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred_label)
    test_acc = accuracy_score(y_test, y_test_pred_label)
    
    return {
        'train_auc': train_auc,
        'train_f1': train_f1,
        'train_acc': train_acc,
        'test_auc': test_auc,
        'test_f1': test_f1,
        'test_acc': test_acc
    }
    
'''
def get_top_features_shap(model, X, output_file="./output/shap_feature_importance.xlsx"):
    """
    Parameters:
    - model: 训练好的模型。
    - X: 用于计算 SHAP 值的特征数据（通常是训练数据）。
    - output_file: 保存 SHAP 排名和 SHAP 值的 Excel 文件名（默认为 "shap_feature_importance.xlsx"）。

    Returns:
    - top_feature_list: 包含按 SHAP 排序的特征名称的列表。
    """
    # 创建解释器
    explainer = shap.Explainer(model.predict, X)
    shap_values = explainer(X)

    # 获取特征重要性
    importance = np.abs(shap_values.values).mean(axis=0)
    # 获取特征名称
    feature_names = X.columns

    # 创建一个 DataFrame 来存储特征及其重要性
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importance,
        'SHAP Value': np.mean(shap_values.values, axis=0)
    })

    # 按重要性排序并提取前 N 个特征
    importance_df = importance_df.sort_values(by='Importance', ascending=False)

    # 将特征的名称保存到列表中
    top_feature_list = importance_df['Feature'].tolist()

    # 保存排名和 SHAP 值到 Excel 文件
    importance_df.to_excel(output_file, index=False)

    print(f"Top features based on SHAP values saved to {output_file}")
    print(importance_df)

    # SHAP 排序图
    #shap.plots.bar(shap_values)

    return top_feature_list

'''

'\ndef get_top_features_shap(model, X, output_file="./output/shap_feature_importance.xlsx"):\n    """\n    Parameters:\n    - model: 训练好的模型。\n    - X: 用于计算 SHAP 值的特征数据（通常是训练数据）。\n    - output_file: 保存 SHAP 排名和 SHAP 值的 Excel 文件名（默认为 "shap_feature_importance.xlsx"）。\n\n    Returns:\n    - top_feature_list: 包含按 SHAP 排序的特征名称的列表。\n    """\n    # 创建解释器\n    explainer = shap.Explainer(model.predict, X)\n    shap_values = explainer(X)\n\n    # 获取特征重要性\n    importance = np.abs(shap_values.values).mean(axis=0)\n    # 获取特征名称\n    feature_names = X.columns\n\n    # 创建一个 DataFrame 来存储特征及其重要性\n    importance_df = pd.DataFrame({\n        \'Feature\': feature_names,\n        \'Importance\': importance,\n        \'SHAP Value\': np.mean(shap_values.values, axis=0)\n    })\n\n    # 按重要性排序并提取前 N 个特征\n    importance_df = importance_df.sort_values(by=\'Importance\', ascending=False)\n\n    # 将特征的名称保存到列表中\n    top_feature_list = importance_df[\'Feature\'].tolist()\n\n    # 保存排名和 SHAP 值到 Excel 文件\n    importa

# RandomForestClassifier

In [84]:
rfc = RandomForestClassifier(random_state=1)
param_grid_rfc = {
    "n_estimators": np.arange(80, 100, 1),
    "criterion": ['gini', 'entropy', 'log_loss'],
    "max_features": np.arange(0.1, 1, 0.1),
    "max_depth": np.arange(1, 13, 1)
}

# 执行超参数搜索
grid_search_rfc = GridSearchCV(rfc, param_grid_rfc, cv=5, scoring='f1', n_jobs=130, verbose=1)
grid_search_rfc.fit(x_train, y_train)
best_model_rfc = grid_search_rfc.best_estimator_

Fitting 5 folds for each of 6480 candidates, totalling 32400 fits


In [85]:
print(x_train.shape[1])
print(f"Best parameters: {grid_search_rfc.best_params_}")
print(f"Best F1 Score (cross-validation): {grid_search_rfc.best_score_}")

results_rfc = evaluate_model(best_model_rfc, x_train, y_train, x_test, y_test)

# 打印结果
print(f"Training AUC: {results_rfc['train_auc']}")
print(f"Training ACC: {results_rfc['train_acc']}")
print(f"Training F1 Score: {results_rfc['train_f1']}")
print(f"External validation AUC: {results_rfc['test_auc']}")
print(f"External validation ACC: {results_rfc['test_acc']}")
print(f"External validation F1 Score: {results_rfc['test_f1']}")

9
Best parameters: {'criterion': 'gini', 'max_depth': 3, 'max_features': 0.4, 'n_estimators': 99}
Best F1 Score (cross-validation): 0.9079577831132454
Training AUC: 0.9283088235294117
Training ACC: 0.8666666666666667
Training F1 Score: 0.9200000000000002
External validation AUC: 0.7407030056036679
External validation ACC: 0.8052631578947368
External validation F1 Score: 0.884012539184953


In [ ]:
#rfc_top_feature_list = get_top_features_shap(best_model_rfc, x_train)

# GradientBoostingClassifier

In [47]:
gbd = GradientBoostingClassifier(random_state = 1)#这个模型比较耗时，参数空间比较大，粗略检索与精细检索结合
param_grid_gbd={
    "learning_rate":np.arange(0.28,0.4,0.01),
    "n_estimators":np.arange(85,95,1),
    "subsample":np.arange(0.72,0.8,0.01),
    "max_features":np.arange(0.01,0.05,0.01),
    "max_depth":np.arange(1,10,1)
}

'''
原始字典
param_grid_gbd={
    "learning_rate":np.arange(0.2,0.4,0.01),
    "n_estimators":np.arange(50,100,1),
    "subsample":np.arange(0.6,0.8,0.01),
    "max_features":np.arange(0.01,0.2,0.01),
    "max_depth":np.arange(1,13,1)
}
'''

# 执行超参数搜索
grid_search_gbd = GridSearchCV(gbd, param_grid_gbd, cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=5), scoring='f1', n_jobs=130, verbose=1)
grid_search_gbd.fit(x_train, y_train.values.ravel())
best_model_gbd = grid_search_gbd.best_estimator_


Fitting 25 folds for each of 38880 candidates, totalling 972000 fits


In [48]:
print(x_train.shape[1])
print(f"Best parameters: {grid_search_gbd.best_params_}")
print(f"Best F1 Score (cross-validation): {grid_search_gbd.best_score_}")

results_gbd = evaluate_model(best_model_gbd, x_train, y_train, x_test, y_test)

# 打印结果
print(f"Training AUC: {results_gbd['train_auc']}")
print(f"Training ACC: {results_gbd['train_acc']}")
print(f"Training F1 Score: {results_gbd['train_f1']}")
print(f"External validation AUC: {results_gbd['test_auc']}")
print(f"External validation ACC: {results_gbd['test_acc']}")
print(f"External validation F1 Score: {results_gbd['test_f1']}")
#rfc_top_feature_list = get_top_features_shap(best_model_gbd, x_train)

Best parameters: {'learning_rate': 0.33000000000000007, 'max_depth': 7, 'max_features': 0.01, 'n_estimators': 86, 'subsample': 0.77}
Best F1 Score (cross-validation): 0.8939860707437493
Training AUC: 1.0
Training ACC: 1.0
Training F1 Score: 1.0
External validation AUC: 0.7164204448972661
External validation ACC: 0.8210526315789474
External validation F1 Score: 0.8944099378881988


# BernoulliNB

In [155]:
bnb=BernoulliNB()
param_grid_bnb={
    "alpha":np.linspace(0,100,100),
    "fit_prior":[True,False],
    "binarize":np.linspace(0,10,100),
    "class_prior":[None]
}

# 执行超参数搜
grid_search_bnb = GridSearchCV(bnb, param_grid_bnb, cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=5), scoring='f1', n_jobs=130, verbose=1)
grid_search_bnb.fit(x_train, y_train)
best_model_bnb = grid_search_bnb.best_estimator_



Fitting 25 folds for each of 20000 candidates, totalling 500000 fits


/home/louhao/.conda/envs/mnf/lib/python3.8/site-packages/sklearn/naive_bayes.py:624: FutureWarning: The default value for `force_alpha` will change to `True` in 1.4. To suppress this warning, manually set the value of `force_alpha`.
  warnings.warn(
/home/louhao/.conda/envs/mnf/lib/python3.8/site-packages/sklearn/naive_bayes.py:633: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(
/home/louhao/.conda/envs/mnf/lib/python3.8/site-packages/sklearn/naive_bayes.py:624: FutureWarning: The default value for `force_alpha` will change to `True` in 1.4. To suppress this warning, manually set the value of `force_alpha`.
  warnings.warn(
/home/louhao/.conda/envs/mnf/lib/python3.8/site-packages/sklearn/naive_bayes.py:633: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(
/home/louhao/.conda/envs/mnf/l

KeyboardInterrupt: 

In [153]:
print(x_train.shape[1])
print(f"Best parameters: {grid_search_bnb.best_params_}")
print(f"Best F1 Score (cross-validation): {grid_search_bnb.best_score_}")
results_bnb = evaluate_model(best_model_bnb, x_train, y_train, x_test, y_test)

# 打印结果
print(f"Training AUC: {results_bnb['train_auc']}")
print(f"Training ACC: {results_bnb['train_acc']}")
print(f"Training F1 Score: {results_bnb['train_f1']}")
print(f"External validation AUC: {results_bnb['test_auc']}")
print(f"External validation ACC: {results_bnb['test_acc']}")
print(f"External validation F1 Score: {results_bnb['test_f1']}")
#rfc_top_feature_list = get_top_features_shap(best_model_bnb, x_train)

8
Best parameters: {'alpha': 0.0, 'binarize': 1.0101010101010102, 'class_prior': None, 'fit_prior': True}
Best F1 Score (cross-validation): 0.8769306459142234
Training AUC: 0.7150101419878296
Training ACC: 0.8066666666666666
Training F1 Score: 0.8781512605042017
External validation AUC: 0.6600441501103753
External validation ACC: 0.7526315789473684
External validation F1 Score: 0.844884488448845


# ExtraTreesClassifier

In [26]:
etc = ExtraTreesClassifier(random_state = 1)
param_grid_etc={
    "n_estimators":np.arange(1,100,1),
    #"max_samples":np.arange(0.01,0.1,0.01),
    "criterion":['gini', 'entropy', 'log_loss'],
    "max_features":np.arange(0.01,0.1,0.01),
    "max_depth":np.arange(1,13,1)
}


# 执行超参数搜索
grid_search_etc = GridSearchCV(etc, param_grid_etc, cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=5), scoring='f1', n_jobs=130)
grid_search_etc.fit(x_train, y_train)
best_model_etc = grid_search_etc.best_estimator_


In [40]:
print(x_train.shape[1])
print(f"Best parameters: {grid_search_etc.best_params_}")
print(f"Best F1 Score (cross-validation): {grid_search_etc.best_score_}")
results_etc = evaluate_model(best_model_etc, x_train, y_train, x_test, y_test)

# 打印结果
print(f"Training AUC: {results_etc['train_auc']}")
print(f"Training ACC: {results_etc['train_acc']}")
print(f"Training F1 Score: {results_etc['train_f1']}")
print(f"External validation AUC: {results_etc['test_auc']}")
print(f"External validation ACC: {results_etc['test_acc']}")
print(f"External validation F1 Score: {results_etc['test_f1']}")
#rfc_top_feature_list = get_top_features_shap(best_model_etc, x_train)

10


NameError: name 'grid_search_etc' is not defined

# LinearSVC

In [58]:
lsvc = LinearSVC(random_state=1, penalty='l2',dual = False, loss = 'squared_hinge')
param_grid_lsvc= {'C':np.arange(0.01,1,0.01)}

# 执行超参数搜索
grid_search_lsvc = GridSearchCV(lsvc, param_grid_lsvc, cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=5), scoring='f1', n_jobs=120, verbose=1)
grid_search_lsvc.fit(x_train, y_train)
best_model_lsvc = grid_search_lsvc.best_estimator_

Fitting 25 folds for each of 99 candidates, totalling 2475 fits


In [59]:
print(x_train.shape[1])
print(f"Best parameters: {grid_search_lsvc.best_params_}")
print(f"Best F1 Score (cross-validation): {grid_search_lsvc.best_score_}")

results_lsvc = evaluate_model(best_model_lsvc, x_train, y_train, x_test, y_test)
# 打印结果
print(f"Training AUC: {results_lsvc['train_auc']}")
print(f"Training ACC: {results_lsvc['train_acc']}")
print(f"Training F1 Score: {results_lsvc['train_f1']}")
print(f"External validation AUC: {results_lsvc['test_auc']}")
print(f"External validation ACC: {results_lsvc['test_acc']}")
print(f"External validation F1 Score: {results_lsvc['test_f1']}")
#rfc_top_feature_list = get_top_features_shap(best_model_lsvc, x_train)

5
Best parameters: {'C': 0.01}
Best F1 Score (cross-validation): 0.8700181723920282
Training AUC: 0.6460446247464503
Training ACC: 0.7766666666666666
Training F1 Score: 0.8723809523809524
External validation AUC: 0.6213278994735949
External validation ACC: 0.7736842105263158
External validation F1 Score: 0.8685015290519877


# SVC

In [79]:
svc = SVC(random_state = 1)
param_grid_svc = {'C':np.arange(100,150,1),
              'gamma':['scale','auto'],
              'kernel':['linear','rbf']}  


# 执行超参数搜索
grid_search_svc = GridSearchCV(svc, param_grid_svc, cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=5), scoring='f1', n_jobs=120, verbose=1)
grid_search_svc.fit(x_train, y_train)
best_model_svc = grid_search_svc.best_estimator_


Fitting 25 folds for each of 200 candidates, totalling 5000 fits


In [80]:
print(x_train.shape[1])
print(f"Best parameters: {grid_search_svc.best_params_}")
print(f"Best F1 Score (cross-validation): {grid_search_svc.best_score_}")

results_svc = evaluate_model(best_model_svc, x_train, y_train, x_test, y_test)

# 打印结果
print(f"Training AUC: {results_svc['train_auc']}")
print(f"Training ACC: {results_svc['train_acc']}")
print(f"Training F1 Score: {results_svc['train_f1']}")
print(f"External validation AUC: {results_svc['test_auc']}")
print(f"External validation ACC: {results_svc['test_acc']}")
print(f"External validation F1 Score: {results_svc['test_f1']}")

#rfc_top_feature_list = get_top_features_shap(best_model_svc, x_train)

5
Best parameters: {'C': 100, 'gamma': 'scale', 'kernel': 'linear'}
Best F1 Score (cross-validation): 0.8721565861400107
Training AUC: 0.563133874239351
Training ACC: 0.7733333333333333
Training F1 Score: 0.8721804511278195
External validation AUC: 0.6281202241467142
External validation ACC: 0.7947368421052632
External validation F1 Score: 0.8856304985337243


# AdaBoostClassifier

In [99]:
abc = AdaBoostClassifier(random_state = 1)
param_grid_abc={
    "n_estimators":np.arange(400,500,1),
    "learning_rate":np.arange(0.1,0.2,0.01),
    "algorithm":['SAMME','SAMME.R']
}

# 执行超参数搜索
grid_search_abc = GridSearchCV(abc, param_grid_abc, cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=5), scoring='f1', n_jobs=130, verbose=1)
grid_search_abc.fit(x_train, y_train)
best_model_abc = grid_search_abc.best_estimator_


Fitting 25 folds for each of 2000 candidates, totalling 50000 fits


In [100]:
print(x_train.shape[1])
print(f"Best parameters: {grid_search_abc.best_params_}")
print(f"Best F1 Score (cross-validation): {grid_search_abc.best_score_}")

results_abc = evaluate_model(best_model_abc, x_train, y_train, x_test, y_test)

# 打印结果
print(f"Training AUC: {results_abc['train_auc']}")
print(f"Training ACC: {results_abc['train_acc']}")
print(f"Training F1 Score: {results_abc['train_f1']}")
print(f"External validation AUC: {results_abc['test_auc']}")
print(f"External validation ACC: {results_abc['test_acc']}")
print(f"External validation F1 Score: {results_abc['test_f1']}")
#rfc_top_feature_list = get_top_features_shap(best_model_abc, x_train)

5
Best parameters: {'algorithm': 'SAMME.R', 'learning_rate': 0.17999999999999997, 'n_estimators': 407}
Best F1 Score (cross-validation): 0.9128160695233936
Training AUC: 0.9887804259634888
Training ACC: 0.94
Training F1 Score: 0.9626556016597512
External validation AUC: 0.7124299541518084
External validation ACC: 0.8421052631578947
External validation F1 Score: 0.9062500000000001


# DecisionTreeClassifier

In [29]:
dtc = DecisionTreeClassifier(random_state = 1)
param_grid_dtc={
    "criterion":['gini','entropy','log_loss'],
    "splitter":['best','random'],
    #"min_samples_leaf":(0.001,0.01,0.001),
    #"min_samples_split":(0.001,0.01,0.001),
    "max_features":np.arange(1,10,1),
    #"max_features":['auto', 'sqrt', 'log2'],#"max_features"要不要设置值应该尝试
   # "max_depth":np.arange(1,13,1)
}

# 执行超参数搜索
grid_search_dtc = GridSearchCV(dtc, param_grid_dtc, cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=5), scoring='f1', n_jobs=120, verbose=1)
grid_search_dtc.fit(x_train, y_train)
best_model_dtc = grid_search_dtc.best_estimator_

Fitting 25 folds for each of 54 candidates, totalling 1350 fits


In [30]:
print(x_train.shape[1])
print(f"Best parameters: {grid_search_dtc.best_params_}")
print(f"Best F1 Score (cross-validation): {grid_search_dtc.best_score_}")
results_dtc = evaluate_model(best_model_dtc, x_train, y_train, x_test, y_test)
# 打印结果
print(f"Training AUC: {results_dtc['train_auc']}")
print(f"Training ACC: {results_dtc['train_acc']}")
print(f"Training F1 Score: {results_dtc['train_f1']}")
print(f"External validation AUC: {results_dtc['test_auc']}")
print(f"External validation ACC: {results_dtc['test_acc']}")
print(f"External validation F1 Score: {results_dtc['test_f1']}")

#rfc_top_feature_list = get_top_features_shap(best_model_dtc, x_train)

5
Best parameters: {'criterion': 'entropy', 'max_features': 5, 'splitter': 'best'}
Best F1 Score (cross-validation): 0.8913535590458502
Training AUC: 1.0
Training ACC: 1.0
Training F1 Score: 1.0
External validation AUC: 0.6355917812871454
External validation ACC: 0.7684210526315789
External validation F1 Score: 0.8552631578947367


# RidgeClassifierCV

In [107]:
rccv =RidgeClassifierCV()
param_grid_rccv={
    "fit_intercept":[True,False],
    "alphas":np.arange(38,40,0.1)
}

# 执行超参数搜索
grid_search_rccv = GridSearchCV(rccv, param_grid_rccv, cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=5), scoring='f1', n_jobs=120, verbose=1)
grid_search_rccv.fit(x_train, y_train)
best_model_rccv = grid_search_rccv.best_estimator_


Fitting 25 folds for each of 40 candidates, totalling 1000 fits


In [108]:
print(x_train.shape[1])
print(f"Best parameters: {grid_search_rccv.best_params_}")
print(f"Best F1 Score (cross-validation): {grid_search_rccv.best_score_}")

results_rccv = evaluate_model(best_model_rccv, x_train, y_train, x_test, y_test)

# 打印结果
print(f"Training AUC: {results_rccv['train_auc']}")
print(f"Training ACC: {results_rccv['train_acc']}")
print(f"Training F1 Score: {results_rccv['train_f1']}")
print(f"External validation AUC: {results_rccv['test_auc']}")
print(f"External validation ACC: {results_rccv['test_acc']}")
print(f"External validation F1 Score: {results_rccv['test_f1']}")

#rfc_top_feature_list = get_top_features_shap(best_model_rccv, x_train)

9
Best parameters: {'alphas': 38.60000000000001, 'fit_intercept': True}
Best F1 Score (cross-validation): 0.872246371914723
Training AUC: 0.7578600405679513
Training ACC: 0.7866666666666666
Training F1 Score: 0.8759689922480621
External validation AUC: 0.5528952283919171
External validation ACC: 0.7842105263157895
External validation F1 Score: 0.876876876876877


# SGDClassifier

In [ ]:
sgdc=SGDClassifier(random_state=1)
param_grid_sgdc={
    "loss":['hinge','log_loss','log','modified_huber','squared_hinge','perceptron','squared_error','huber','epsilon_insensitive','squared_epsilon_insensitive'],
    "penalty":['l2','l1','elasticnet'],
    "alpha":np.arange(0.01,0.1,0.01),
    "l1_ratio":np.arange(0.1,0.5,0.1),
    "fit_intercept":[True,False],
    "learning_rate":['optimal','invscaling','adaptive']
    #"learning_rate":np.arange(0.01,1,0.01)
}

# 执行超参数搜索
grid_search_sgdc = GridSearchCV(sgdc, param_grid_sgdc, cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=5), scoring='f1', n_jobs=120, verbose=1)
grid_search_sgdc.fit(x_train, y_train)
best_model_sgdc = grid_search_sgdc.best_estimator_


In [ ]:
print(f"Best parameters: {grid_search_sgdc.best_params_}")
print(f"Best F1 Score (cross-validation): {grid_search_sgdc.best_score_}")
results_sgdc = evaluate_model(best_model_sgdc, x_train, y_train, x_test, y_test)

# 打印结果
print(f"Training AUC: {results_sgdc['train_auc']}")
print(f"Training ACC: {results_sgdc['train_acc']}")
print(f"Training F1 Score: {results_sgdc['train_f1']}")
print(f"External validation AUC: {results_sgdc['test_auc']}")
print(f"External validation ACC: {results_sgdc['test_acc']}")
print(f"External validation F1 Score: {results_sgdc['test_f1']}")

#rfc_top_feature_list = get_top_features_shap(best_model_sgdc, x_train)

# Perceptron

In [119]:
p=Perceptron(random_state=1)
param_grid_p={
    "penalty":['l2','l1','elasticnet'],
    "alpha":np.arange(0.000001,0.0001,0.000001),
    "fit_intercept":[True,False]
}


# 执行超参数搜索
grid_search_p = GridSearchCV(p, param_grid_p, cv=5, scoring='f1', n_jobs=120, verbose=1)
grid_search_p.fit(x_train, y_train)
best_model_p = grid_search_p.best_estimator_


Fitting 5 folds for each of 600 candidates, totalling 3000 fits


In [120]:
print(x_train.shape[1])
print(f"Best parameters: {grid_search_p.best_params_}")
print(f"Best F1 Score (cross-validation): {grid_search_p.best_score_}")
results_p = evaluate_model(best_model_p, x_train, y_train, x_test, y_test)

# 打印结果
print(f"Training AUC: {results_p['train_auc']}")
print(f"Training ACC: {results_p['train_acc']}")
print(f"Training F1 Score: {results_p['train_f1']}")
print(f"External validation AUC: {results_p['test_auc']}")
print(f"External validation ACC: {results_p['test_acc']}")
print(f"External validation F1 Score: {results_p['test_f1']}")
#rfc_top_feature_list = get_top_features_shap(best_model_p, x_train)

8
Best parameters: {'alpha': 3.5e-05, 'fit_intercept': True, 'penalty': 'elasticnet'}
Best F1 Score (cross-validation): 0.840948532043811
Training AUC: 0.6947261663286005
Training ACC: 0.7166666666666667
Training F1 Score: 0.8098434004474273
External validation AUC: 0.6138563423331636
External validation ACC: 0.7157894736842105
External validation F1 Score: 0.8223684210526315


# PassiveAggressiveClassifier

In [140]:
pac=PassiveAggressiveClassifier(random_state=1)
param_grid_pac={
    "C":np.arange(0.001,1.0,0.001),
    "fit_intercept":[True,False]
}

# 执行超参数搜索
grid_search_pac = GridSearchCV(pac, param_grid_pac, cv=5, scoring='f1', n_jobs=120, verbose=1)
grid_search_pac.fit(x_train, y_train)
best_model_pac = grid_search_pac.best_estimator_


Fitting 5 folds for each of 1998 candidates, totalling 9990 fits


In [141]:
print(x_train.shape[1])
print(f"Best parameters: {grid_search_pac.best_params_}")
print(f"Best F1 Score (cross-validation): {grid_search_pac.best_score_}")
results_pac = evaluate_model(best_model_pac, x_train, y_train, x_test, y_test)

# 打印结果
print(f"Training AUC: {results_pac['train_auc']}")
print(f"Training ACC: {results_pac['train_acc']}")
print(f"Training F1 Score: {results_pac['train_f1']}")
print(f"External validation AUC: {results_pac['test_auc']}")
print(f"External validation ACC: {results_pac['test_acc']}")
print(f"External validation F1 Score: {results_pac['test_f1']}")

#rfc_top_feature_list = get_top_features_shap(best_model_pac, x_train)

5
Best parameters: {'C': 0.060000000000000005, 'fit_intercept': True}
Best F1 Score (cross-validation): 0.8754674733224135
Training AUC: 0.7351673427991885
Training ACC: 0.7833333333333333
Training F1 Score: 0.8747591522157997
External validation AUC: 0.608762098828324
External validation ACC: 0.7947368421052632
External validation F1 Score: 0.8835820895522387
